In [3]:
import re
import requests
import pandas as pd
import os
import soccerdata as sd
import numpy as np

from PIL import Image
from bs4 import BeautifulSoup

[06/06/24 11:55:57] INFO     No custom team name replacements found. You can configure these in       ]8;id=984385;file://C:\Users\Dilhan\anaconda3\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=60227;file://C:\Users\Dilhan\anaconda3\Lib\site-packages\soccerdata\_config.py#87\87]8;;\
                             C:\Users\Dilhan\soccerdata\config\teamname_replacements.json.                         

                    INFO     Custom league dict loaded from                                          ]8;id=348532;file://C:\Users\Dilhan\anaconda3\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=905311;file://C:\Users\Dilhan\anaconda3\Lib\site-packages\soccerdata\_config.py#180\180]8;;\
                             C:\Users\Dilhan\soccerdata\config\league_dict.json.                                   

In [1]:
def scraping_header(url, headers=None):
    """
    Scrape content from the provided URL with optional custom headers and return the parsed HTML content of the page.
    """
    if headers is None:
        headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raises an HTTPError for bad responses
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None
    return BeautifulSoup(response.content, 'html.parser')

def compare_teams(df1, df2, column='Team', df1_name='DF1', df2_name='DF2'):
    """
    Compare specified columns between two dataframes to find unique data in each and diplay the difference from each dataframe.
    """
    if column not in df1 or column not in df2:
        raise ValueError(f"The column '{column}' must exist in both dataframes.")
    
    # Get unique values from each dataframe
    df1_unique = df1[~df1[column].isin(df2[column])][column].drop_duplicates().sort_values(ascending=False).reset_index(drop=True)
    df2_unique = df2[~df2[column].isin(df1[column])][column].drop_duplicates().sort_values(ascending=False).reset_index(drop=True)

    if df1_unique.empty and df2_unique.empty:
        print(f"All {column} entries are present and correctly named in both {df1_name} and {df2_name}.")
        return None
    
    comparison_df = pd.DataFrame({
        f'{df1_name} Unique': df1_unique,
        f'{df2_name} Unique': df2_unique
    }).fillna('')
    
    return comparison_df


def align_and_merge_teams(df1, df2, column='Team'):
    """
    Merges the two dataframes according to the team
    """
    
    if column not in df1.columns or column not in df2.columns:
        raise ValueError(f"The column '{column}' must exist in both dataframes.")
        
    df1_sorted = df1.sort_values(by=column).reset_index(drop=True)
    df2_sorted = df2.sort_values(by=column).reset_index(drop=True)
    
    mapping_dict = dict(zip(df1_sorted[column], df2_sorted[column]))
    
    df1_aligned = df1.copy()
    df1_aligned[column] = df1[column].map(mapping_dict).fillna(df1[column])
    
    missing_teams = set(df1_aligned[column]) - set(df2_sorted[column])
    if missing_teams:
        print("Missing teams in alignment:", missing_teams)
    else:
        print("All", column, "are present and correctly named.")
        
    merged_df = pd.merge(df1_aligned, df2_sorted, on=column, how='left', suffixes=('', '_drop'))
    merged_df.drop([col for col in merged_df.columns if '_drop' in col], axis=1, inplace=True)
    
    return merged_df

## Import players salary

In [3]:
def import_players_salary():
    soup = scraping_header("https://mlsplayers.org/resources/salary-guide")

    table = soup.find('table', {'id': 'salary-report'})
    rows = table.find_all('tr')

    data = []
    for row in rows[1:]:  # Skip the header row
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append(cols)  # Get rid of empty values

    df_players_salary = pd.DataFrame(data, columns=['FirstName', 'LastName', 'Team', 'Position', 'SalaryBase ($)', 'SalaryGuaranteed ($)']) 
    
    df_players_salary.isnull().sum() # Check if there is null values
    
    df_players_salary['SalaryBase ($)'] = pd.to_numeric(df_players_salary['SalaryBase ($)'].str.replace('[$,]', '', regex=True))
    df_players_salary['SalaryGuaranteed ($)'] = pd.to_numeric(df_players_salary['SalaryGuaranteed ($)'].str.replace('[$,]', '', regex=True))
    
    df_players_salary.to_csv('datasets/players_salary.csv', index=False)
    return df_players_salary

In [4]:
# Function no longer used in our case, as the site has been updated to show salaries for 2024. It's the 2023 salaries 
# we're interested in at the moment

# df_players_salary = import_players_salary() 

In [5]:
df_players_salary = pd.read_csv('datasets/players_salary.csv') # read the csv file created with 2023 data
df_players_salary.head()

,FirstName,LastName,Team,Position,SalaryBase ($),SalaryGuaranteed ($)
0,Luis,Abram,Atlanta United,D,556364.0,695977.0
1,Lalas,Abubakar,Colorado Rapids,D,665000.0,702125.0
2,Nicolás,Acevedo,New York City FC,D-M,230000.0,274800.0
3,Alonso,Aceves,Chicago Fire,D,330000.0,368900.0
4,Ifunanyachi,Achara,Houston Dynamo,F,85444.0,85444.0


## Create Team expenses table

In [6]:
salary_column = 'SalaryGuaranteed ($)'
total_salary_by_club = df_players_salary.groupby('Team')[salary_column].sum().sort_values(ascending=False).reset_index()
total_salary_by_club = total_salary_by_club[total_salary_by_club['Team'] != "Major League Soccer"]

## Add colors Team

In [7]:
soup = scraping_header("https://teamcolorcodes.com/soccer/mls-team-color-codes/")
teams = soup.find_all('a', class_='team-button')

team_colors = {}
for team in teams:
    name = team.get_text().strip()
    main_color = team['style'].split(';')[0].split(':')[-1].strip()
    secondary_color = team['style'].split(';')[1].split(':')[-1].replace('4px solid ', '').strip()
    team_colors[name] = {'Main Color': main_color, 'Secondary Color': secondary_color}

df_teams = pd.DataFrame.from_dict(team_colors, orient='index').reset_index()
df_teams.columns = ['Team', 'MainColor', 'SecondaryColor']

In [8]:
manual_teams = {
    'Nashville SC': {'Main Color': '#ECE83A', 'Secondary Color': '#1F1646'},
    'St. Louis City SC': {'Main Color': '#E0004D', 'Secondary Color': '#0C2340'},
    'Charlotte FC': {'Main Color': '#000000', 'Secondary Color': '#1A85C8'},
}
team_name_mappingSC = {
    "Vancouver Whitecaps": "Vancouver Whitecaps FC",
    "Minnesota United": "Minnesota United FC",
    "LAFC": "Los Angeles FC",
    "DC United": "D.C. United",
    "Columbus Crew": "Columbus Crew SC",
    "Inter Miami": "Inter Miami CF",
    "FC Cincinnati": "Cincinnati FC",
    "CF Montreal": "Montreal Impact",
}

In [9]:
team_colors.update(manual_teams)
df_teams = pd.DataFrame.from_dict(team_colors, orient='index').reset_index()
df_teams.columns = ['Team', 'MainColor', 'SecondaryColor']
total_salary_by_club['Team'] = total_salary_by_club['Team'].replace(team_name_mappingSC)

In [10]:
compare_teams(total_salary_by_club, df_teams, df1_name='Team Salary', df2_name='Team Colors')
df_teams = align_and_merge_teams(total_salary_by_club, df_teams)

All Team entries are present and correctly named in both Team Salary and Team Colors.
All Team are present and correctly named.


In [11]:
df_teams.to_csv('datasets/MLS_team_colors.csv', index=False)

## Final Table

In [12]:
mls_tables = pd.read_html('https://en.wikipedia.org/wiki/Template:2023_Major_League_Soccer_season_table')
mls_table = mls_tables[0]
mls_table = mls_table.rename(columns={mls_table.columns[1]: 'Team'})
mls_table = mls_table.drop(mls_table.columns[-1], axis=1)

mls_table['GD'] = mls_table['GD'].replace({'−': '-'}, regex=True)
mls_table['GD'] = pd.to_numeric(mls_table['GD'])

mls_table.head()

,Pos,Team,Pld,W,L,T,GF,GA,GD,Pts
0,1,FC Cincinnati (S),34,20,5,9,57,39,18,69
1,2,Orlando City SC,34,18,7,9,55,39,16,63
2,3,Columbus Crew (C),34,16,9,9,67,46,21,57
3,4,St. Louis City SC,34,17,12,5,62,45,17,56
4,5,Philadelphia Union,34,15,9,10,57,41,16,55


In [13]:
compare_teams(mls_table, df_teams, df1_name='MLS Table', df2_name='Team Colors')

,MLS Table Unique,Team Colors Unique
0,Vancouver Whitecaps FC (V),Vancouver Whitecaps FC
1,Inter Miami CF (L),Montreal Impact
2,Houston Dynamo FC (U),Inter Miami CF
3,FC Cincinnati (S),Houston Dynamo
4,Columbus Crew (C),Columbus Crew SC
5,Chicago Fire FC,Cincinnati FC
6,CF Montréal,Chicago Fire
7,Atlanta United FC,Atlanta United


In [14]:
team_name_mappingSC = {
    "FC Cincinnati (S)": "Cincinnati FC",
    "CF Montréal": "Montreal Impact",
}
mls_table['Team'] = mls_table['Team'].replace(team_name_mappingSC)
mls_table = align_and_merge_teams(mls_table, df_teams)

All Team are present and correctly named.


## Teams logo

In [15]:
folder_destination = 'datasets/Teamslogo/'

In [16]:
soup = scraping_header("https://www.sportslogos.net/teams/list_by_league/9/major_league_soccer/mls/logos/")

if not os.path.exists(folder_destination):
    os.makedirs(folder_destination)

# Download logos
logo_wall = soup.find('ul', class_='logoWall')
logos = logo_wall.find_all('li', style=True)
for logo in logos:
    team_name = logo.find('a')['title'].replace(' Logos', '')
    logo_path = logo.find('img')['src']
    logo_response = requests.get(logo_path, stream=True)
    extension = os.path.splitext(logo_path)[1]
    filename = f"{team_name}{extension}"
    file_path = os.path.join(folder_destination, filename)

    if logo_response.status_code == 200:
        with open(file_path, 'wb') as f:
            for chunk in logo_response.iter_content(chunk_size=128):
                f.write(chunk)

team_logos = []
for filename in os.listdir(folder_destination):
    if filename.endswith('.gif'):
        base_filename = filename[:-4].rstrip()
        new_filename = f"{base_filename}.png"
        
        original_filepath = os.path.join(folder_destination, filename)
        
        with Image.open(original_filepath) as img:
            new_filepath = os.path.join(folder_destination, new_filename)
            img.save(new_filepath)
        
        os.remove(original_filepath)
        
        team_name = base_filename.strip()
        image_path = os.path.join(folder_destination, new_filename)
        
        team_logos.append({'Team': team_name, 'Logo path': image_path})

logos_df = pd.DataFrame(team_logos)

In [17]:
compare_teams(mls_table, logos_df, df1_name='MLS Table', df2_name='Logo Table')

,MLS Table Unique,Logo Table Unique
0,St. Louis City SC,St Louis City SC
1,Montreal Impact,LAFC
2,Los Angeles FC,Inter Miami C.F.
3,Inter Miami CF,FC Cincinnati
4,Cincinnati FC,Club de Foot Montreal
5,Atlanta United,Atlanta United FC


In [18]:
team_name_mappingSC = {
    "Club de Foot Montreal": "Montreal Impact",
}
logos_df['Team'] = logos_df['Team'].replace(team_name_mappingSC) 
logos_df = align_and_merge_teams(logos_df, mls_table)
logos_df = align_and_merge_teams(total_salary_by_club, logos_df)

All Team are present and correctly named.
All Team are present and correctly named.


### Enhance and adjust image display

In [19]:
def remove_white_background(img_path, output_path):
    """
    Changes the background of the image from white to transparent
    """
    with Image.open(img_path) as img:
        # Convert the image to RGBA mode to access the alpha channel
        img = img.convert("RGBA")
        datas = img.getdata()
        newData = []
        
        for item in datas:
            if item[0] > 220 and item[1] > 220 and item[2] > 220:
                newData.append((255, 255, 255, 0))
            else:
                newData.append(item)

        img.putdata(newData)
        bbox = img.getbbox()
        img_cropped = img.crop(bbox)
        img_cropped.save(output_path)

In [20]:
def remove_white_background(img_path, output_path):
    """
    Changes the background of the image from white to transparent
    """
    with Image.open(img_path) as img:
        # Convert the image to RGBA mode to access the alpha channel
        img = img.convert("RGBA")
        # Convert to numpy array for faster processing https://medium.com/codex/say-goodbye-to-loops-in-python-and-welcome-vectorization-e8b0172b9581 
        data = np.array(img)  

        # Set white or near-white pixels to transparent
        white = np.all(data[:, :, :3] >= 220, axis=-1)
        data[white, -1] = 0

        img_cropped = Image.fromarray(data)
        bbox = img_cropped.getbbox()
        img_cropped = img_cropped.crop(bbox)
        img_cropped.save(output_path)

In [21]:
for filename in os.listdir(folder_destination):
    if filename.endswith('.png'):
        file_path = os.path.join(folder_destination, filename)
        remove_white_background(file_path, file_path)

In [22]:
logos_df.sort_values(by='Pos', ascending=True)

,Team,SalaryGuaranteed ($),Logo path,Pos,Pld,W,L,T,GF,GA,GD,Pts,MainColor,SecondaryColor
20,Cincinnati FC,14348728.0,datasets/Teamslogo/Colorado Rapids.png,1,34,20,5,9,57,39,18,69,#003087,#FE5000
28,Orlando City SC,9642918.0,datasets/Teamslogo/Orlando City SC.png,2,34,18,7,9,55,39,16,63,#61259E,#FFE293
13,Columbus Crew SC,15313747.0,datasets/Teamslogo/D.C. United.png,3,34,16,9,9,67,46,21,57,#231f20,#FEF200
26,St. Louis City SC,11186787.0,datasets/Teamslogo/St Louis City SC.png,4,34,17,12,5,62,45,17,56,#E0004D,#0C2340
24,Philadelphia Union,13419282.0,datasets/Teamslogo/Philadelphia Union.png,5,34,15,9,10,57,41,16,55,#002D55,#B38707
10,New England Revolution,17013587.0,datasets/Teamslogo/New England Revolution.png,6,34,15,9,10,58,46,12,55,#E51938,#002B5C
6,Seattle Sounders FC,19185352.0,datasets/Teamslogo/Seattle Sounders FC.png,7,34,14,9,11,41,32,9,53,#236192,#658D1B
3,Los Angeles FC,20841863.0,datasets/Teamslogo/LAFC.png,8,34,14,10,10,54,39,15,52,#000000,#C39e6d
8,Houston Dynamo,17459127.0,datasets/Teamslogo/Houston Dynamo.png,9,34,14,11,9,51,38,13,51,#F68712,#8DC6ED
5,Atlanta United,19847184.0,datasets/Teamslogo/Atlanta United FC.png,10,34,13,9,12,66,53,13,51,#80000A,#A19060


In [23]:
logos_df.to_csv('datasets/MLS_23_table.csv', index=False)

## Scraping Fbref

In [24]:
soup = scraping_header("https://fbref.com/en/comps/22/2023/2023-Major-League-Soccer-Stats")
table = soup.find('table', {'id': 'stats_squads_standard_for'})
rows = table.find_all('tr')

data = []
for row in rows[2:]:  # Skip the headers row
    cols = row.find_all('th')
    cols = cols + row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)  # Get rid of empty values
    
df_team_FBref_for = pd.DataFrame(data, columns=['Team', '# Pl', 'Age', 'Poss', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 
                                            'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 
                                            'PrgC', 'PrgP', 'Gls/90', 'Ast/90', 'G+A/90', 'G-PK/90', 'G+A-PK/90', 'xG/90', 
                                            'xAG/90', 'xG+xAG/90', 'npxG/90', 'npxG+xAG/90'])

In [25]:
table = soup.find('table', {'id': 'stats_squads_standard_against'})
rows = table.find_all('tr')

data = []
for row in rows[2:]:  # Skip the headers row
    cols = row.find_all('th')
    cols = cols + row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)  # Get rid of empty values
    
df_team_FBref_against = pd.DataFrame(data, columns=['Team', '# Pl', 'Age vs', 'Poss vs', 'MP', 'Starts', 'Min', '90s', 'Gls vs', 'Ast vs', 
                                            'G+A vs', 'G-PK vs', 'PK vs', 'PKatt vs', 'CrdY vs', 'CrdR vs', 'xG vs', 'npxG vs', 'xAG vs', 'npxG+xAG vs', 
                                            'PrgC vs', 'PrgP vs', 'Gls/90 vs', 'Ast/90 vs', 'G+A/90 vs', 'G-PK/90 vs', 'G+A-PK/90 vs', 'xG/90 vs', 
                                            'xAG/90 vs', 'xG+xAG/90 vs', 'npxG/90 vs', 'npxG+xAG/90'])
df_team_FBref_against['Team'] = df_team_FBref_against['Team'].str.replace("vs ", "")

In [26]:
df_team_Fbref = align_and_merge_teams(df_team_FBref_for, df_team_FBref_against)

All Team are present and correctly named.


In [27]:
compare_teams(df_team_Fbref, logos_df, df1_name='Fbref Table', df2_name='Logo Table')

,Fbref Table Unique,Logo Table Unique
0,Vancouver W'caps,Vancouver Whitecaps FC
1,St. Louis,St. Louis City SC
2,Sporting KC,Sporting Kansas City
3,Seattle,Seattle Sounders FC
4,SJ Earthquakes,San Jose Earthquakes
5,Rapids,Real Salt Lake
6,RSL,Philadelphia Union
7,Philadelphia,Orlando City SC
8,Orlando City,New York Red Bulls
9,Nashville,New York City FC


In [28]:
team_name_mappingFL = {
    "NY Red Bulls": "New York Red Bulls",
    "NYCFC": "New York City FC",
    "CF Montréal": "Montreal Impact",
}
df_team_Fbref['Team'] = df_team_Fbref['Team'].replace(team_name_mappingFL)
df_team_Fbref = align_and_merge_teams(logos_df, df_team_Fbref)
df_team_Fbref.drop(columns=["Pld", "GF", "GA", "GD"])

All Team are present and correctly named.


,Team,SalaryGuaranteed ($),Logo path,Pos,W,L,T,Pts,MainColor,SecondaryColor,...,PrgP vs,Gls/90 vs,Ast/90 vs,G+A/90 vs,G-PK/90 vs,G+A-PK/90 vs,xG/90 vs,xAG/90 vs,xG+xAG/90 vs,npxG/90 vs
0,LA Galaxy,39419071.0,datasets/Teamslogo/Inter Miami C.F..png,27,9,18,7,34,#231F20,#F7B5CD,...,1382,1.91,1.32,3.24,1.79,3.12,1.60,1.20,2.80,1.51
1,Toronto FC,31672886.0,datasets/Teamslogo/Toronto FC.png,29,4,20,10,22,#AB1E2D,#3f4743,...,1537,1.68,1.09,2.76,1.59,2.68,1.59,1.14,2.73,1.51
2,LAFC,25038692.0,datasets/Teamslogo/LA Galaxy.png,26,8,14,12,36,#00245d,#ffd200,...,1211,1.12,0.82,1.94,1.03,1.85,1.12,0.79,1.91,1.00
3,Minnesota Utd,20841863.0,datasets/Teamslogo/LAFC.png,8,14,10,10,52,#000000,#C39e6d,...,1521,1.47,1.09,2.56,1.44,2.53,1.40,1.08,2.48,1.38
4,Crew,20415828.0,datasets/Teamslogo/Chicago Fire.png,24,10,14,10,40,#121F48,#B3272D,...,970,1.32,0.94,2.26,1.24,2.18,1.21,0.94,2.15,1.13
5,Atlanta Utd,19847184.0,datasets/Teamslogo/Atlanta United FC.png,10,13,9,12,51,#80000A,#A19060,...,1238,1.53,1.24,2.76,1.44,2.68,1.35,1.03,2.38,1.26
6,Seattle,19185352.0,datasets/Teamslogo/Seattle Sounders FC.png,7,14,9,11,53,#236192,#658D1B,...,1299,0.88,0.71,1.59,0.85,1.56,0.95,0.70,1.65,0.92
7,Austin,19038551.0,datasets/Teamslogo/Austin FC.png,25,10,15,9,39,#000000,#00b140,...,1268,1.59,1.09,2.68,1.50,2.59,1.47,1.05,2.51,1.40
8,Inter Miami,17459127.0,datasets/Teamslogo/Houston Dynamo.png,9,14,11,9,51,#F68712,#8DC6ED,...,1293,1.50,1.21,2.71,1.47,2.68,1.53,1.21,2.74,1.51
9,FC Dallas,17302163.0,datasets/Teamslogo/FC Cincinnati.png,23,10,14,10,40,#ee1a39,#231f20,...,1330,0.97,0.68,1.65,0.94,1.62,1.08,0.77,1.85,1.01


In [29]:
def convert_columns_to_numeric(df, columns):
    """
    Convert specified columns in the dataframe to numeric
    """
    df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')
    return df

def calculate_diff_columns(df, cols_to_diff):
    """
    Calculate the difference between pairs of columns and create a new column for each pair
    """
    for col1, col2 in cols_to_diff:
        diff_col_name = f"Diff_{col1}"
        df[diff_col_name] = df[col1] - df[col2]
    return df

In [30]:
# Convert specified columns to numeric
columns_to_convert = ['Gls', 'Gls vs', 'xG', 'xG vs']
df_team_Fbref = convert_columns_to_numeric(df_team_Fbref, columns_to_convert)

# Calculate differences and create new columns
columns_to_difference = [('Gls', 'xG'), ('Gls vs', 'xG vs')]
df_team_Fbref = calculate_diff_columns(df_team_Fbref, columns_to_difference)

In [31]:
df_team_Fbref

,Team,SalaryGuaranteed ($),Logo path,Pos,Pld,W,L,T,GF,GA,...,Ast/90 vs,G+A/90 vs,G-PK/90 vs,G+A-PK/90 vs,xG/90 vs,xAG/90 vs,xG+xAG/90 vs,npxG/90 vs,Diff_Gls,Diff_Gls vs
0,LA Galaxy,39419071.0,datasets/Teamslogo/Inter Miami C.F..png,27,34,9,18,7,41,54,...,1.32,3.24,1.79,3.12,1.60,1.20,2.80,1.51,-1.2,10.5
1,Toronto FC,31672886.0,datasets/Teamslogo/Toronto FC.png,29,34,4,20,10,26,59,...,1.09,2.76,1.59,2.68,1.59,1.14,2.73,1.51,-5.3,3.0
2,LAFC,25038692.0,datasets/Teamslogo/LA Galaxy.png,26,34,8,14,12,51,67,...,0.82,1.94,1.03,1.85,1.12,0.79,1.91,1.00,-2.6,0.0
3,Minnesota Utd,20841863.0,datasets/Teamslogo/LAFC.png,8,34,14,10,10,54,39,...,1.09,2.56,1.44,2.53,1.40,1.08,2.48,1.38,-10.4,2.4
4,Crew,20415828.0,datasets/Teamslogo/Chicago Fire.png,24,34,10,14,10,39,51,...,0.94,2.26,1.24,2.18,1.21,0.94,2.15,1.13,5.3,3.7
5,Atlanta Utd,19847184.0,datasets/Teamslogo/Atlanta United FC.png,10,34,13,9,12,66,53,...,1.24,2.76,1.44,2.68,1.35,1.03,2.38,1.26,15.3,6.0
6,Seattle,19185352.0,datasets/Teamslogo/Seattle Sounders FC.png,7,34,14,9,11,41,32,...,0.71,1.59,0.85,1.56,0.95,0.70,1.65,0.92,-10.1,-2.2
7,Austin,19038551.0,datasets/Teamslogo/Austin FC.png,25,34,10,15,9,49,55,...,1.09,2.68,1.50,2.59,1.47,1.05,2.51,1.40,5.5,4.1
8,Inter Miami,17459127.0,datasets/Teamslogo/Houston Dynamo.png,9,34,14,11,9,51,38,...,1.21,2.71,1.47,2.68,1.53,1.21,2.74,1.51,2.8,-1.2
9,FC Dallas,17302163.0,datasets/Teamslogo/FC Cincinnati.png,23,34,10,14,10,45,49,...,0.68,1.65,0.94,1.62,1.08,0.77,1.85,1.01,3.2,-3.6


In [32]:
df_team_Fbref.to_csv('datasets/squad_stats_FBref.csv', index=False)

In [33]:
fbref = sd.FBref(leagues=['US-MLS'], seasons=['2324'])
player_season_stats = fbref.read_player_season_stats(stat_type="standard")
player_season_stats.head()

[06/04/24 16:51:09] INFO     Saving cached data to C:\Users\Dilhan\soccerdata\data\FBref              ]8;id=416337;file://C:\Users\Dilhan\anaconda3\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=398215;file://C:\Users\Dilhan\anaconda3\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

C:\Users\Dilhan\anaconda3\Lib\site-packages\soccerdata\_common.py:498: UserWarning: Season id "2021" is ambiguous: interpreting as "20-21"
  warnings.warn(msg, stacklevel=1)


nation pos age  born Playing Time  \
                                                                       MP   
league season team        player                                            
US-MLS 2324   Atlanta Utd Ajani Fortune    TRI  MF  20  2002           14   
                          Amar Sejdic      USA  MF  26  1996           23   
                          Andrew Gutman    USA  DF  26  1996           18   
                          Brad Guzan       USA  GK  38  1984           27   
                          Brooks Lennon    USA  DF  25  1997           33   

                                                           Performance      \
                                        Starts   Min   90s         Gls Ast   
league season team        player                                             
US-MLS 2324   Atlanta Utd Ajani Fortune      3   374   4.2           0   1   
                          Amar Sejdic       14  1061  11.8           0   0   
                          Andrew Gutman     16  1385  15.4           3   2   
                          Brad Guzan        27  2430  27.0           0   0   
                          Brooks Lennon     32  2881  32.0           4   9   

                                         ... Per 90 Minutes                    \
                                         ...            Gls   Ast   G+A  G-PK   
league season team        player         ...                                    
US-MLS 2324   Atlanta Utd Ajani Fortune  ...            0.0  0.24  0.24   0.0   
                          Amar Sejdic    ...            0.0   0.0   0.0   0.0   
                          Andrew Gutman  ...           0.19  0.13  0.32  0.19   
                          Brad Guzan     ...            0.0   0.0   0.0   0.0   
                          Brooks Lennon  ...           0.12  0.28  0.41  0.12   

                                                                         \
                                        G+A-PK    xG   xAG xG+xAG  npxG   
league season team        player                                          
US-MLS 2324   Atlanta Utd Ajani Fortune   0.24  0.18  0.03   0.21  0.18   
                          Amar Sejdic      0.0  0.03  0.02   0.05  0.03   
                          Andrew Gutman   0.32  0.11  0.14   0.24  0.11   
                          Brad Guzan       0.0   0.0   0.0    0.0   0.0   
                          Brooks Lennon   0.41  0.08  0.16   0.24  0.08   

                                                  
                                        npxG+xAG  
league season team        player                  
US-MLS 2324   Atlanta Utd Ajani Fortune     0.21  
                          Amar Sejdic       0.05  
                          Andrew Gutman     0.24  
                          Brad Guzan         0.0  
                          Brooks Lennon     0.24  

[5 rows x 33 columns]

In [34]:
def clean_and_rename_columns(df):
    """
    Reset index of the dataframe, remove unnecessary columns and rename to simplify
    """
    df = df.reset_index()
    df.drop(columns=['league', 'season'], errors='ignore', inplace=True)
    
    new_columns = []
    for col in df.columns:
        if col[1] == '':
            new_columns.append(col[0])

        elif col[0] == 'Per 90 Minutes':
            new_columns.append(col[1] + '/90')
        
        else:
            new_columns.append(col[1])
    
    df.columns = new_columns    
    return df

In [35]:
player_season_stats = clean_and_rename_columns(player_season_stats)
player_season_stats.columns

C:\Users\Dilhan\AppData\Local\Temp\ipykernel_42280\4291252125.py:6: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns=['league', 'season'], errors='ignore', inplace=True)


Index(['team', 'player', 'nation', 'pos', 'age', 'born', 'MP', 'Starts', 'Min',
       '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG',
       'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls/90', 'Ast/90',
       'G+A/90', 'G-PK/90', 'G+A-PK/90', 'xG/90', 'xAG/90', 'xG+xAG/90',
       'npxG/90', 'npxG+xAG/90'],
      dtype='object')

In [36]:
player_season_stats.head()

,team,player,nation,pos,age,born,MP,Starts,Min,90s,...,Gls/90,Ast/90,G+A/90,G-PK/90,G+A-PK/90,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90
0,Atlanta Utd,Ajani Fortune,TRI,MF,20,2002,14,3,374,4.2,...,0.0,0.24,0.24,0.0,0.24,0.18,0.03,0.21,0.18,0.21
1,Atlanta Utd,Amar Sejdic,USA,MF,26,1996,23,14,1061,11.8,...,0.0,0.0,0.0,0.0,0.0,0.03,0.02,0.05,0.03,0.05
2,Atlanta Utd,Andrew Gutman,USA,DF,26,1996,18,16,1385,15.4,...,0.19,0.13,0.32,0.19,0.32,0.11,0.14,0.24,0.11,0.24
3,Atlanta Utd,Brad Guzan,USA,GK,38,1984,27,27,2430,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Atlanta Utd,Brooks Lennon,USA,DF,25,1997,33,32,2881,32.0,...,0.12,0.28,0.41,0.12,0.41,0.08,0.16,0.24,0.08,0.24


In [37]:
player_season_stats.to_csv('datasets/player_stats_FBref.csv', index=False)

## Import WhoScored Data

In [38]:
def clean_columns(df, drop_columns):
    """
    Cleans the DataFrame by resetting the index and dropping specified columns.
    """
    df = df.reset_index()
    df.drop(columns=drop_columns, errors='ignore', inplace=True)
    return df

def calculate_missed_games(df):
    """
    Calculates the total and unique missed games by club and the player who missed the most games.
    """
    total_missed = df.groupby('team')['game_id'].count().rename('total_missed_games')
    unique_missed = df.groupby('team')['player'].nunique().rename('unique_players_missing')
    most_missed = df.groupby(['team', 'player'])['game_id'].count().reset_index(name='missed_games')
    most_missed = most_missed.loc[most_missed.groupby('team')['missed_games'].idxmax()]

    missed_summary = pd.concat([total_missed, unique_missed], axis=1).reset_index()
    missed_summary = missed_summary.merge(most_missed, on='team', how='left')

    missed_summary.rename(columns={'player': 'player_most_missed', 'missed_games': 'most_missed_games'}, inplace=True)
    return missed_summary

def add_opponent_info(df, schedule_df):
    """
    Calculates the number of missing players for opposing teams
    """
    df = df.merge(schedule_df, on='game_id', how='left')
    df['opponent'] = df.apply(lambda row: row['away_team'] if row['team'] == row['home_team'] else row['home_team'], axis=1)
    return df

In [53]:
def import_whoScored_data():
    ws = sd.WhoScored(leagues="US-MLS", seasons=2324)
    mls_schedule = ws.read_schedule()
    missing_players = ws.read_missing_players()
    
    mls_schedule = clean_columns(mls_schedule, ['league', 'season', 'game', 'url', 'stage'])
    missing_players = clean_columns(missing_players, ['league', 'season', 'game'])
    
    missing_players = add_opponent_info(missing_players, mls_schedule)
    total_missed_by_club = calculate_missed_games(missing_players)
    # Calculate missed games by opponent
    missed_by_opponent = (
        missing_players.groupby('opponent')['game_id']
        .count()
        .reset_index(name='total_missed_games_by_opponent')
        .rename(columns={'opponent': 'team'})
    )
    total_missed_by_club = total_missed_by_club.merge(
        missed_by_opponent, on='team', how='left'
    ).fillna({'total_missed_games_by_opponent': 0})
    
    total_missed_by_club.sort_values(by='total_missed_games', ascending=False)
    
    total_missed_by_club.to_csv('datasets/missed_games_by_club.csv', index=False)
    
    return total_missed_by_club

In [52]:
# The soccerdata library is no longer functional for scraping whoScored, so the import of the csv file generated when it was 
# functional is performed while waiting for the anomaly to be corrected

# total_missed_by_club = total_missed_by_club()

In [68]:
total_missed_by_club = pd.read_csv('datasets/missed_games_by_club.csv')
total_missed_by_club.rename(columns={'team': 'Team'}, inplace=True)
total_missed_by_club.head() 

,Team,total_missed_games,unique_players_missing,player_most_missed,most_missed_games,total_missed_games_by_opponent
0,Atlanta United,85,19,Osvaldo Alonso,22,105
1,Austin FC,76,16,Zan Kolmanic,24,108
2,CF Montreal,99,17,Romell Quioto,17,89
3,Charlotte FC,113,21,GuzmÃ¡n Corujo,15,121
4,Chicago Fire FC,119,18,Federico Navarro,19,109


In [57]:
compare_teams(df_team_Fbref, total_missed_by_club, df1_name='Fbref Table', df2_name='Missed Table')

,Fbref Table Unique,Missed Table Unique
0,Vancouver W'caps,Vancouver Whitecaps
1,St. Louis,St. Louis City
2,Sporting KC,Sporting Kansas City
3,Seattle,Seattle Sounders FC
4,SJ Earthquakes,San Jose Earthquakes
5,Rapids,Real Salt Lake
6,RSL,Philadelphia Union
7,Philadelphia,New England Revolution
8,Nashville,Nashville SC
9,NE Revolution,Minnesota United


In [69]:
team_name_mappingWF = {
    "Rapids": "Colorado Rapids",
    "Nashville": "Nashville SC",
    "Montreal Impact": "CF Montreal",
    "Nashville": "Nashville SC",
    "Fire": "Chicago Fire FC",
}
df_team_Fbref['Team'] = df_team_Fbref['Team'].replace(team_name_mappingWF)
stats_mls = align_and_merge_teams(df_team_Fbref, total_missed_by_club)

All Team are present and correctly named.


In [78]:
stats_mls.to_csv('datasets/stats_mls.csv', index=False)